# Terraform
Let's shape the world

In [ ]:
#| default_exp terrain

### To Do's

1. Create a new resource group for triangles
1. Fix contour so it only does the high side

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq

In [ ]:
#| export
sys.path.append(".")
sys.path.append("..")

from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, rt, StyleDemo

from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexRegion, HexWrapper, HexPosition, PrimitiveDemo, hexBackground

In [ ]:
PrimitiveDemo().demoDrawGrid()

In [ ]:
??HexGrid.from_bounds


```python
@classmethod
def from_bounds(cls, bounds: MapRect, radius: float=25, style: StyleCSS=StyleCSS("Hex")) -> 'HexGrid':
    """Create grid to fill a bounding rectangle."""

    nRows = int(bounds.dimensons.height / radius)
    nCols = int(bounds.dimensons.width / radius)
    offset = MapCord(bounds.origin.x - radius, bounds.origin.y - radius)

    return cls(nRows, nCols, radius, style, offset)
```

**File:** `~/HexMagic/HexMagic/primitives.py`

In [ ]:
preview(PrimitiveDemo().demoHexBackground())

#| export
### Helpers

In [ ]:
#| export
class TerraDemo:
    def __init__(self):
        self.help = ""

### Don't know much geography

In [ ]:
#| export
@dataclass
class GeoBounds:
    """Geographic bounds in lat/lon."""
    lat_min: float
    lat_max: float
    lon_min: float
    lon_max: float
    
    @property
    def lat_center(self):
        return (self.lat_min + self.lat_max) / 2
    
    @property
    def lon_center(self):
        return (self.lon_min + self.lon_max) / 2
    
    @property
    def lat_span(self):
        return self.lat_max - self.lat_min
    
    @property
    def lon_span(self):
        return self.lon_max - self.lon_min

In [ ]:
#| export
@dataclass
class ClimatePreset:
    """Preset parameters for different climate zones."""
    name: str
    lat_range: tuple[float, float]
    base_temp_range: tuple[float, float]  # (south/low, north/high)
    
    # Smith-Barstad precipitation parameters
    wind_speed: float = 10.0
    wind_dir: float = 270.0
    precip_base: float = 0.1
    nm: float = 0.01
    hw: float = 2000.0
    cw: float = 0.002
    conv_time: float = 2000.0
    fall_time: float = 2000.0
    
    # Temperature parameters
    lapse_rate: float = 6.5
    
    # Precipitation thresholds for 5 dot size levels (mm/year)
    # Creates 5 ranges: <bin[0], bin[0]-bin[1], bin[1]-bin[2], bin[2]-bin[3], >bin[3]
    precip_bins: tuple[float, float, float, float] = (400, 800, 1500, 2500)
    
    # Description
    description: str = ""
    
    def get_precip_level(self, precip_mm: float) -> int:
        """Convert precipitation to dot size level (0-4) for this climate."""
        for i, threshold in enumerate(self.precip_bins):
            if precip_mm < threshold:
                return i
        return len(self.precip_bins)  # Highest level (4)
    
    def encode(self) -> str:
        """Encode climate preset to a single line string."""
        bins_str = ','.join(map(str, self.precip_bins))
        parts = [
            self.name,
            f"{self.lat_range[0]},{self.lat_range[1]}",
            f"{self.base_temp_range[0]},{self.base_temp_range[1]}",
            str(self.wind_speed),
            str(self.wind_dir),
            str(self.precip_base),
            str(self.nm),
            str(self.hw),
            str(self.cw),
            str(self.conv_time),
            str(self.fall_time),
            str(self.lapse_rate),
            bins_str,  # Add precip_bins
            self.description.replace('|', '//')  # escape pipe chars
        ]
        return '|'.join(parts)
    
    @staticmethod
    def decode(s: str) -> 'ClimatePreset':
        """Decode climate preset from string."""
        parts = s.split('|')
        lat_range = tuple(map(float, parts[1].split(',')))
        temp_range = tuple(map(float, parts[2].split(',')))
        precip_bins = tuple(map(float, parts[12].split(',')))
        
        return ClimatePreset(
            name=parts[0],
            lat_range=lat_range,
            base_temp_range=temp_range,
            wind_speed=float(parts[3]),
            wind_dir=float(parts[4]),
            precip_base=float(parts[5]),
            nm=float(parts[6]),
            hw=float(parts[7]),
            cw=float(parts[8]),
            conv_time=float(parts[9]),
            fall_time=float(parts[10]),
            lapse_rate=float(parts[11]),
            precip_bins=precip_bins,
            description=parts[13].replace('//', '|') if len(parts) > 13 else ""
        )


## Terrain
Stores the information about a map

In [ ]:
#| export
class Terrain:

    def __init__(self,
    bounds:MapRect, #our canvas size, 
    path: str = "sample.svg", #where are we going to export
    radius: float = 25.0, #default hex size,
    fields:[str] = ["rainfall","tempLow","tempHigh"],
    colorLevels:[StyleCSS] = StyleCSS.elevations(),
    seaLevel:StyleCSS =  StyleCSS("sealevel",fill ="#81b1e1ff"),
    elevationDelta:float = 90,
    geo:GeoBounds = None,
    climate:ClimatePreset = None
   

    ):
        self.colorLevels = colorLevels
        self.seaLevel = seaLevel
        self.hexGrid = HexGrid.from_bounds(bounds, radius=radius, style=seaLevel)
        self.hexGrid.builder.add_style(seaLevel)
        self.path = path
        self.climate = climate

        for  x in self.colorLevels: 
            hover = StyleCSS("hover",fill=x.desaturate(),cursor="pointer" )
            #hover = StyleCSS("hover",fill="#007fff",cursor="pointer" )
            x.customize(hover)
            self.hexGrid.builder.add_style(x)
           
        self.elevationDelta = elevationDelta
        self.elevations = np.zeros(len(self.hexGrid.hexes))
        self.fields = {}
        self.geo = geo
        for field in  fields:
            self.fields[field] = np.zeros(len(self.hexGrid.hexes))
       
    @property
    def builder(self):
        return self.hexGrid.builder

    def colorMap(self):
        numHeights = len(self.colorLevels)
        if numHeights < 1:
            return 0
        for i in range(len(self.elevations)):
            height = int(self.elevations[i]  / self.elevationDelta)
            
            aLevel =  self.seaLevel
            if self.elevations[i]  > 0:
                if height < numHeights:
                    aLevel = self.colorLevels[height]
                else:
                    aLevel = self.colorLevels[-1]
            self.hexGrid.hexes[i].style = aLevel
            


    def textElevations(self):
        grid = self.hexGrid 
        nRows = grid.nRows
        nCols =  grid.nCols
        
        i = 0
        for row in range(nRows):
        
            line = "|"
            for col in range(nCols):
                line += f" { int(self.elevations[i]/100):3}"
                i += 1

            print(line + " |")

    def textStyle(self):
        grid = self.hexGrid 
        nRows = grid.nRows
        nCols =  grid.nCols
        
        i = 0
        for row in range(nRows):
        
            line = "|"
            for col in range(nCols):
                line += f" { self.hexGrid.hexes[i].style.name[:3]:3}"
                i += 1

            print(line + " |")
         
    
    def encode(self):
        grid = self.hexGrid 
        nRows = grid.nRows
        nCols = grid.nCols
        ret = f"radius:{self.hexGrid.radius}\n"
        ret += f"size:{self.hexGrid.nRows}^{self.hexGrid.nCols}\n"
        ret += f"path:{self.path}\n"
        field_names = '\t'.join(self.fields.keys())
        ret += f"fields:{field_names}\n"
        
        
        # Add geobounds if present
        if self.geo is not None:
            ret += f"geo:{self.geo.lat_min},{self.geo.lat_max},{self.geo.lon_min},{self.geo.lon_max}\n"
        
        # Add climate if present
        if self.climate is not None:
            ret += f"climate:{self.climate.encode()}\n"
        
        ret += f"+data:\n"
        i = 0
        for row in range(nRows):
            line = []
            for col in range(nCols):
                csv_parts = [str(self.elevations[i])]
                for fieldName in self.fields.keys():
                    csv_parts.append(str(self.fields[fieldName][i]))
                line.append(','.join(csv_parts))
                i += 1
            ret += "\t".join(line) + "\n"
        ret += f"-data:\n"
        
        return ret


    @staticmethod
    def decode(s: str):
        """Parse a formatted string and create a Terrain from it."""
        lines = s.strip().split('\n')
        
        # Parse header
        radius = None
        path = None
        geo = None
        climate = None  # New: store climate preset
        inGrid = False
        nRow = 0
        nCol = 0
        fieldNames = ["rainfall", "tempLow", "tempHigh"]
        elevations = []
        fieldData = {}
        
        for line in lines:
            if not inGrid:
                if ':' not in line:
                    continue
                val = line.split(':', 1)[1].strip()  # Use maxsplit=1 for climate line
                
                if line.startswith('radius:'):
                    radius = float(val)
                elif line.startswith('size:'):
                    size = val.split('^')
                    nRow = int(size[0])
                    nCol = int(size[1])
                elif line.startswith('path:'):
                    path = val
                elif line.startswith('fields:'):
                    fieldNames = val.split('\t')
                elif line.startswith('geo:'):
                    coords = val.split(',')
                    geo = GeoBounds(
                        lat_min=float(coords[0]),
                        lat_max=float(coords[1]),
                        lon_min=float(coords[2]),
                        lon_max=float(coords[3])
                    )
                elif line.startswith('climate:'):
                    climate = ClimatePreset.decode(val)
                elif line.startswith('+data:'):
                    inGrid = True
                    fieldData = {name: [] for name in fieldNames}
            else:
                if line.startswith('-data:'):
                    break
                else:
                    cells = line.split('\t')
                    for cell in cells:
                        values = cell.split(',')
                        elevations.append(float(values[0]))
                        for i, fieldName in enumerate(fieldNames):
                            fieldData[fieldName].append(float(values[i + 1]))
        
        # Create Terrain object
        bounds = MapRect(MapCord(0, 0), MapSize(nRow * radius, nCol * radius))
        retTerrain = Terrain(bounds=bounds, path=path, radius=radius, climate=climate, geo=geo)
        retTerrain.hexGrid.nRows = nRow
        retTerrain.hexGrid.nCols = nCol
        retTerrain.hexGrid.adjustRadius(radius)
        
        retTerrain.elevations = np.array(elevations)
        retTerrain.fields = {name: np.array(fieldData[name]) for name in fieldNames}
        
        return retTerrain




In [ ]:
#| export
@patch
def elevationLevel(self: Terrain, idx):
    """Return the elevation level (integer) for a hex by dividing by elevationDelta."""
    if self.elevations[idx] > 0:
        return int(self.elevations[idx] / self.elevationDelta)
    return -1

@patch
def export(self:Terrain,wrapper:HexWrapper = None):
    if wrapper is None:
        wrapper = HexWrapper()

    testStr = self.encode()
    with open(self.path, 'w') as f:
        f.write(testStr)

@patch
def ring(self:Terrain,center,radius=1):
    if radius < 1:
        return [center]
    ring_hexpositions = HexPosition(0, 0, 0).ring(radius)
    ring_indices = [self.hexGrid.hexposition_to_index(hp, center) for hp in ring_hexpositions]
    return [i for i in ring_indices if i >= 0]  # Filter out-of-bounds

In [ ]:
#| export
@patch
def sanFran(self:TerraDemo):
    with open("data/templates/san_francisco.txt", "r") as f:
        myGrid = Terrain.decode(f.read())
    return myGrid

@patch
def agincourt(self:TerraDemo):
    with open("data/templates/agincourt.txt", "r") as f:
        myGrid = Terrain.decode(f.read())
    return myGrid

In [ ]:

TerraDemo().sanFran().textElevations()

|   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   0   0   0   0   0   1   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 |
|   0   

Can you fix?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the error, the problem is in the `Terrain.__init__` method on line 18. You're calling:

```python
self.hexGrid = HexGrid.from_bounds(bounds, seaLevel, radius=radius)
```

But `HexGrid.from_bounds` expects `radius` as the second parameter (not `seaLevel`). You're passing `seaLevel` as a positional argument where `radius` should be, and then also passing `radius` as a keyword argument - that's why it says "got multiple values for argument 'radius'".

The fix is to swap the order. Change line 18 to:

```python
self.hexGrid = HexGrid.from_bounds(bounds, radius=radius, style=seaLevel)
```

This matches the signature you showed earlier where `from_bounds` takes `bounds`, `radius`, and `style` as parameters.

In [ ]:
#| export
@patch
def demoSanFran(self:TerraDemo):
    sampleMap = self.agincourt()
    sampleMap.elevationDelta = 20
    colors = StyleCSS.seaborn("rocket",13)
    sampleMap.colorLevels = colors
    
    for color in colors:
        sampleMap.hexGrid.builder.add_style(color)

    sampleMap.hexGrid.adjustRadius(5)
    sampleMap.colorMap()
    
    sampleMap.hexGrid.update()
    return sampleMap.hexGrid.builder.show()



In [ ]:

TerraDemo().demoSanFran()

In [ ]:
#| export
@patch
def elevation_borders(self: Terrain)->str:
    """Draw hex borders with stroke color/width based on elevation."""
    
    border_svg = ""
    border_styles = {}
    elevation_styles = StyleCSS.seaborn("YlOrBr", levels=13)[2:8]
    for i, style in enumerate(elevation_styles):
        fill = style.properties["fill"]
        border_styles[i] = (f"stroke:{fill}; stroke-width:{1.5}; opacity:{0.5 + 0.5 * i}", "{style.name}_tacoBell")
    #border_styles[0] =  ("stroke:#5a7a8f; stroke-width:0.5; opacity:0.3", "coastal")

    # Define border styles by elevation level
    oldborder_styles = {
        0: ("stroke:#5a7a8f; stroke-width:0.5; opacity:0.3", "coastal"),  # Sea level - subtle
        1: ("stroke:#7d6e5d; stroke-width:0.8; opacity:0.4", "lowland"),
        2: ("stroke:#8b7d6b; stroke-width:1.0; opacity:0.5", "hills"),
        3: ("stroke:#9d8b73; stroke-width:1.2; opacity:0.6", "highlands"),
        4: ("stroke:#a89579; stroke-width:1.5; opacity:0.7", "mountains"),
        5: ("stroke:red; stroke-width:3; opacity:0.8", "peaks"), # was b5a085
    }
    
    for i, hex in enumerate(self.hexGrid.hexes):
        elev_level = self.elevationLevel(i)
        
        if elev_level >= 0:  # Skip ocean
            style_str, style_name = border_styles.get(
                min(elev_level, 5), 
                border_styles[5]
            )
            
            # Draw just the border (no fill)
            border_svg += f'<polygon points="'
            for point in hex.vertices():
                border_svg += f"{point.x:.0f},{point.y:.0f} "
            border_svg += f'" style="fill:none; {style_str}"/>\n'

    return border_svg

In [ ]:
#| export
@patch
def add_elevation_borders(self: Terrain, layer_name="elevation_borders"):
    """Draw hex borders with stroke color/width based on elevation."""
    
    border_svg = self.elevation_borders()
    
    self.hexGrid.builder.adjust(layer_name, border_svg)
    return self

In [ ]:
@patch
def demoSanFranBod(self:TerraDemo):
    sampleMap = self.agincourt()
    sampleMap.elevationDelta = 20
    colors = StyleCSS.seaborn("crest",7)
    sampleMap.colorLevels = colors
    
    for color in colors:
        sampleMap.hexGrid.builder.add_style(color)

    sampleMap.hexGrid.adjustRadius(5)
    sampleMap.colorMap()
    
    sampleMap.hexGrid.update()
    sampleMap.add_elevation_borders()
    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoSanFranBod()

In [ ]:
#| export
@patch
def tiny(self:TerraDemo):
    mySize = MapSize(120,120)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=30,path = "tiny.svg")

    adjustments =   np.zeros(len(sampleMap.elevations))
    baseHeight = 600
    origin = 5
  
    for i in sampleMap.ring(origin,2):
        adjustments[i] = baseHeight/4
    
    for i in sampleMap.ring(origin,1):
        adjustments[i] = baseHeight/2

    adjustments[origin] = baseHeight

    sampleMap.elevations += adjustments
    return sampleMap

In [ ]:
#| export
@patch
def demoTiny(self:TerraDemo):
    

    sampleMap = self.tiny()


    print(sampleMap.encode())
    print("=======")

    sampleMap.hexGrid.adjustRadius(5)
    print(sampleMap.encode())

    print("++===++")
    sampleMap.hexGrid.adjustRadius(50)
    sampleMap.colorMap()
    
    sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()






In [ ]:
TerraDemo().demoTiny()

radius:30
size:4^4
path:tiny.svg
fields:rainfall	tempLow	tempHigh
+data:
150.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
300.0,0.0,0.0,0.0	600.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
150.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
150.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0	0.0,0.0,0.0,0.0
-data:

radius:5
size:4^4
path:tiny.svg
fields:rainfall	tempLow	tempHigh
+data:
150.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
300.0,0.0,0.0,0.0	600.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
150.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	300.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0
150.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0	150.0,0.0,0.0,0.0	0.0,0.0,0.0,0.0
-data:

++===++


### Plot Thickens

Based on the code context, here's a summary of how overlay plotting works in this terrain visualization system:

**Overlay Plotting Summary**

The overlay system allows you to visualize additional data layers (like rainfall, temperature) on top of the base terrain elevation map using SVG patterns.

**Key Components:**

1. **Pattern Generation** (`ballPatterns`): Creates a set of SVG pattern definitions, typically representing different intensity levels. Each pattern is a scaled circle whose radius increases with the level index.

2. **Data Binning** (`rainfall_selector_np`): Converts continuous data values into discrete pattern indices using `np.digitize`. For example, rainfall amounts are binned into categories (0-5, 5-12, 12-24, 24-48, 48+ inches).

3. **Overlay Application** (`makeOverlay`): 
   - Adds pattern definitions to the SVG builder
   - Iterates through each hex cell
   - Applies the appropriate pattern as a fill based on the data index
   - Creates polygon elements with `url(#pattern_id)` fills

**Workflow:**
```python
# 1. Prepare your data field
rainfall_data = np.random.uniform(-4, 60, len(terrain.elevations))
terrain.fields['rainfall'] = rainfall_data

# 2. Bin the data into pattern indices
pattern_indices = rainfall_selector_np(rainfall_data)

# 3. Generate patterns and apply overlay
patterns = ballPatterns(5)  # 5 intensity levels
terrain.makeOverlay(pattern_indices, patterns)
```

The result is a dual-layer visualization: colored hexes showing elevation beneath patterned overlays showing your additional data field.

## Terain Patterns

In [ ]:
#| export

@patch
def colorRegions(self:Terrain,regions:[HexRegion],fills:[str])->str:
    testBody = ""
    grid = self.hexGrid

    addSet = set()
    for cI, region in enumerate(regions):
        print(f" region {cI} has hexes {len(region.hexes)}")
        
        if cI >= len(fills):
            cI = len(fills) - 1

        fill = fills[cI]
        for i in region.hexes:
            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"

            testBody += "\t" +  ret + "\n"

    return testBody

In [ ]:
#| export
@patch
def styleRegion(self:Terrain,region:HexRegion,style:StyleCSS)->str:
    testBody = ""
    grid = self.hexGrid

    
    for i in region.hexes:
        ret = "<polygon points=\""
        hex = grid.hexes[i]
        for point in hex.vertices():
            ret += f"{point.x:.0f},{point.y:.0f} "
        ret += f"\" class=\"{style.name}\""
        ret += "/>"
        testBody += "\t" +  ret + "\n"

    return testBody

### Uplift

In [ ]:
#| export
@patch
def cone(self:Terrain, center, adjusted, num_rings, variability=0.0):
    """Create a volcano-like elevation pattern.
    
    Args:
        center: index of center hex
        adjusted: peak height at center
        num_rings: how many rings outward to affect
        variability: amount of random variation (0 = none)
    """
    adjustments = np.zeros(len(self.elevations))
    
    # Center hex
    center_variation = random.uniform(-variability, variability) if variability > 0 else 0
    adjustments[center] = adjusted + center_variation
    
    # Process each ring outward
    for ring_num in range(1, num_rings + 1):
        ring_indices = self.ring(center, ring_num)
        
        for idx in ring_indices:
            # Calculate base height using smooth curve
            distance_ratio = ring_num / num_rings
            base_height = adjusted * (1 - distance_ratio) ** 2
            
            # Get neighbors from inner ring
            hp = self.hexGrid.index_to_hexposition(idx, center)
            neighbor_positions = hp.ring(1)
            neighbor_indices = [self.hexGrid.hexposition_to_index(nhp, center) 
                               for nhp in neighbor_positions]
            
            # Find which neighbors are in the inner ring
            inner_ring_indices = set(self.ring(center, ring_num - 1))
            adjacent_inner = [ni for ni in neighbor_indices 
                            if ni >= 0 and ni in inner_ring_indices]
            
            # Calculate variability based on adjacent inner ring hexes
            if adjacent_inner and variability > 0:
                avg_inner = np.mean([adjustments[i] for i in adjacent_inner])
                variation = random.uniform(-variability, variability)
                # Blend the variation with the inner ring average
                height_variation = (avg_inner - base_height) * 0.3 + variation
            else:
                height_variation = random.uniform(-variability, variability) if variability > 0 else 0
            
            adjustments[idx] = base_height + height_variation
    
    return adjustments


In [ ]:
#| export
@patch
def volcano(self:Terrain, center, adjusted, num_rings, variability=0.0, initial_threshold=0.8):
    """Create a volcano-like elevation pattern with threshold-based coastlines.
    
    Args:
        center: index of center hex
        adjusted: peak height at center
        num_rings: how many rings outward to affect
        variability: amount of random variation (0 = none)
        initial_threshold: starting threshold (decreases each ring outward)
    """
    adjustments = np.zeros(len(self.elevations))
    
    # Center hex (always applied)
    center_variation = random.uniform(-variability, variability) if variability > 0 else 0
    adjustments[center] = adjusted + center_variation
    
    # Process each ring outward
    for ring_num in range(1, num_rings + 1):
        ring_indices = self.ring(center, ring_num)
        
        # Calculate threshold for this ring (decreases as we go outward)
        #threshold = initial_threshold * (1 - ring_num / num_rings)
        threshold = initial_threshold * (ring_num / num_rings)

        
        for idx in ring_indices:
            # Check if this hex passes the threshold
            if random.random() > threshold:
                # Calculate base height using smooth curve
                distance_ratio = ring_num / num_rings
                base_height = adjusted * (1 - distance_ratio) ** 2
                
                # Get neighbors from inner ring
                hp = self.hexGrid.index_to_hexposition(idx, center)
                neighbor_positions = hp.ring(1)
                neighbor_indices = [self.hexGrid.hexposition_to_index(nhp, center) 
                                   for nhp in neighbor_positions]
                
                # Find which neighbors are in the inner ring
                inner_ring_indices = set(self.ring(center, ring_num - 1))
                adjacent_inner = [ni for ni in neighbor_indices 
                                if ni >= 0 and ni in inner_ring_indices]
                
                # Calculate variability based on adjacent inner ring hexes
                if adjacent_inner and variability > 0:
                    avg_inner = np.mean([adjustments[i] for i in adjacent_inner])
                    variation = random.uniform(-variability, variability)
                    # Blend the variation with the inner ring average
                    height_variation = (avg_inner - base_height) * 0.3 + variation
                else:
                    height_variation = random.uniform(-variability, variability) if variability > 0 else 0
                
                adjustments[idx] = base_height + height_variation
    
    return adjustments


In [ ]:
#| export
@patch
def demoVolcano(self:TerraDemo):

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=15,path = "volcano.svg")
    
    sampleMap.elevations += sampleMap.volcano(center=267,adjusted=500,num_rings=6,variability=0.5,initial_threshold=0.4)
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()


In [ ]:
TerraDemo().demoVolcano()

In [ ]:
#| export
@patch
def find_peaks(self:Terrain, k, min_height, exclusion_radius=1):
    """Find k largest peaks above min_height with exclusion zones.
    
    Args:
        k: maximum number of peaks to find
        min_height: minimum elevation threshold
        exclusion_radius: how many rings around each peak to exclude
    
    Returns:
        List of indices of peaks (up to k peaks)
    """
 
    # Create max heap of all hexes above threshold
    # Use negative elevation for max heap behavior
    candidates = [(-self.elevations[i], i) 
                  for i in range(len(self.elevations)) 
                  if self.elevations[i] >= min_height]
    
    heapq.heapify(candidates)
    
    peaks = []
    blocked = set()
    
    while len(peaks) < k and candidates:
        neg_elevation, idx = heapq.heappop(candidates)
        
        # Skip if this hex is blocked
        if idx in blocked:
            continue
        
        # Add this peak
        peaks.append(idx)
        
        # Block this peak and surrounding hexes
        for radius in range(exclusion_radius + 1):
            for blocked_idx in self.ring(idx, radius):
                blocked.add(blocked_idx)
    
    return peaks


In [ ]:
#| export
@patch
def demoPeakSan(self:TerraDemo):
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    for peak in peaks:
        sampleMap.elevations[peak] = 900
    #sampleMap.elevationDelta = 20
    sampleMap.hexGrid.adjustRadius(5)
    sampleMap.colorMap()
    
    sampleMap.hexGrid.update()
    return sampleMap.hexGrid.builder.show()




In [ ]:
TerraDemo().demoPeakSan()

In [ ]:
#| export
@patch
def lowest_neighbor(self:Terrain, idx):
    """Find the lowest neighbor of idx, or None if idx is a local minimum."""
    neighbors = self.ring(idx)
    current_elev = self.elevations[idx]
    
    lowest_idx = None
    lowest_elev = current_elev
    
    for n in neighbors:
        if self.elevations[n] < lowest_elev:
            lowest_elev = self.elevations[n]
            lowest_idx = n
    
    return lowest_idx

In [ ]:
@patch
def demoRegion(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    print(region.perimeter())
    print("===================")

    ring_indices = sampleMap.ring(start, 1)
    for idx in ring_indices:
        region.hexes.add(idx)
        print(len(region.perimeter()))

    print("===================")
    print(region.perimeter())

    
    

TerraDemo().demoRegion()

[(290,336), (290,348), (280,354), (270,348), (270,336), (280,330)]
10
12
14
16
18
18
[(280,366), (270,372), (259,366), (259,354), (249,348), (249,336), (259,330), (301,354), (301,366), (290,372), (311,336), (311,348), (301,330), (280,318), (259,318), (270,312), (301,318), (290,312)]


In [ ]:
#| export
@patch
def dot(self:Terrain, 
    points:[MapCord],
    style=StyleCSS("dataPoint",fill="#aa6092ff",stroke="#4bc151ff",stroke_width=0.5),
    layerName="circles",
    radius = 3 ):
   
    # Draw circles at perimeter points

    circle_layer = ""
    for point in points:
        circle_layer += f"""\t<circle cx="{point.x}" cy="{point.y}" r="{radius}" class="{style.name}"/>\n"""
  
    
    self.builder.add_style(style)
    self.builder.adjust(layerName, circle_layer)
    
    

In [ ]:
#| export
@patch
def fillRegion(self:Terrain, region:HexRegion):
    """Fill a region with a style."""
    
    fill_layer = ""

    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    
    flowData = np.zeros(len(self.hexGrid.hexes)) - 1

    for i in region.hexes:
        #print(i)
        flowData[i] = 1

    flowData = [int(x) for x in flowData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(self)
    patterns = patternGen.ballDensity(3,fills=fills)  # 5 levels
    self.makeOverlay(flowData, patterns)
    

In [ ]:
#| export
@patch
def colorRegions(self:Terrain,regions:[HexRegion],fills:[str])->str:
    testBody = ""
    grid = self.hexGrid

    addSet = set()
    for cI, region in enumerate(regions):
        print(f" region {cI} has hexes {len(region.hexes)}")
        
        if cI >= len(fills):
            cI = len(fills) - 1

        fill = fills[cI]
        for i in region.hexes:
            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"

            testBody += "\t" +  ret + "\n"

    return testBody

@patch
def demoRegion(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    
    ring_indices = sampleMap.ring(start)
    for idx in ring_indices:
        region.hexes.add(idx)
        
    ring_indices = sampleMap.ring(start,2)
    for idx in ring_indices:
        region.hexes.add(idx)

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    sampleMap.dot(region.perimeter())
    sampleMap.fillRegion(region)
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()


TerraDemo().demoRegion()

In [ ]:
#| export
@patch
def demoRegion(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    
    ring_indices = sampleMap.ring(start)
    for idx in ring_indices:
        region.hexes.add(idx)
        
    ring_indices = sampleMap.ring(start,2)
    for idx in ring_indices:
        region.hexes.add(idx)

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    #sampleMap.dot(region.perimeter())
    #sampleMap.fillRegion(region)

    pathstyle=StyleCSS("boundaryPath",fill="none",stroke="#ba3ca3ff",stroke_width=3)
    paths = region.trace_perimeter(style=pathstyle)
    sampleMap.builder.add_style(pathstyle)
    
    
    pathLayer = ""
    for path in paths:
        cl = path.closed()
        w = cl.make_windy(iterations=2, offset_factor=0.2)
        pathLayer += w.svg()
        
    sampleMap.builder.adjust("perimeter_path", pathLayer)
    
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()
 



In [ ]:
TerraDemo().demoRegion()

## Honey I shrunk the map

In [ ]:
#| export
@patch
def _sample_indexes(self:Terrain, rows, cols, fraction):
    new_rows = int(rows * fraction)
    new_cols = int(cols * fraction)
    arr = np.arange(rows*cols)
    
    # Create new array filled with large values
    new_arr = np.full(new_rows * new_cols, np.inf)
    
    # Map each original element to new position
    for r in range(rows):
        for c in range(cols):
            new_r = int(r * fraction)
            new_c = int(c * fraction)
            if new_r < new_rows and new_c < new_cols:
                idx = new_r * new_cols + new_c
                new_arr[idx] = int(min(new_arr[idx], arr[r * cols + c]))
    
    return new_arr, new_rows, new_cols

In [ ]:
#| export
@patch
def convolution(self: Terrain, field, shape:[HexPosition], fraction, weights = None, method='weighted_avg'):
    """sample any field array using hex ring sampling.
    
    Args:
        field: numpy array of values to downsample (same length as hexes)
        shape: List of HexPosition offsets defining the sampling pattern (e.g., from ring() or spiral())
        new_grid: HexGrid for the downsampled result
        weights: Optional weights for each position in shape
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        Downsampled array matching new_grid size
    """
    
    grid = self.hexGrid
    destIndex, new_rows, new_cols = self._sample_indexes( self.hexGrid.nRows, self.hexGrid.nCols, fraction)
    # Calculate new grid size
    new_size = new_rows * new_cols
    new_field = np.zeros(new_size)
    
    # Default weights if none provided
    if weights is None:
        weights = np.ones(len(shape))
    
    # Iterate through new grid
    for new_idx, sampling_index in enumerate(destIndex):
        values = []
        valid_weights = []

        for i, offset in enumerate(shape):
            old_idx = self.hexGrid.hexposition_to_index(offset, int(sampling_index))
            if 0 <= old_idx < len(field):
                values.append(field[old_idx])
                valid_weights.append(weights[i])
        
        # If no valid samples, use 0
        if len(values) == 0:
            new_field[new_idx] = 0
            continue
            
        values = np.array(values)
        valid_weights = np.array(valid_weights)
        
        # Apply aggregation method
        if method == 'weighted_avg':
            new_field[new_idx] = np.average(values, weights=valid_weights)
        elif method == 'max':
            new_field[new_idx] = np.max(values)
        elif method == 'min':
            new_field[new_idx] = np.min(values)
        elif method == 'mode':
            # For categorical data - weighted mode
            unique_vals = np.unique(values)
            best_val = unique_vals[0]
            best_weight = 0
            for val in unique_vals:
                val_weight = np.sum(valid_weights[values == val])
                if val_weight > best_weight:
                    best_weight = val_weight
                    best_val = val
            new_field[new_idx] = best_val
    
    return new_field, new_rows, new_cols

#| export
def sample_array( rows, cols, fraction):
    new_rows = int(rows * fraction)
    new_cols = int(cols * fraction)
    arr = np.arange(rows*cols)
    
    # Create new array filled with large values
    new_arr = np.full(new_rows * new_cols, np.inf)
    
    # Map each original element to new position
    for r in range(rows):
        for c in range(cols):
            new_r = int(r * fraction)
            new_c = int(c * fraction)
            if new_r < new_rows and new_c < new_cols:
                idx = new_r * new_cols + new_c
                new_arr[idx] = int(min(new_arr[idx], arr[r * cols + c]))
    
    return new_arr, new_rows, new_cols
    
@patch
def convolution(self: Terrain, field, shape:[HexPosition], new_grid: HexGrid, weights = None, method='weighted_avg'):
    """sample any field array using hex ring sampling.
    
    Args:
        field: numpy array of values to downsample (same length as hexes)
        shape: List of HexPosition offsets defining the sampling pattern (e.g., from ring() or spiral())
        new_grid: HexGrid for the downsampled result
        weights: Optional weights for each position in shape
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        Downsampled array matching new_grid size
    """
    
    # Calculate new grid size
    new_size = new_grid.nCols * new_grid.nRows
    new_field = np.zeros(new_size)
    
    # Calculate scaling factor between old and new grids
    scale_x = self.hexGrid.nCols / new_grid.nCols
    scale_y = self.hexGrid.nRows / new_grid.nRows
    
    # Default weights if none provided
    if weights is None:
        weights = np.ones(len(shape))
    
    # Iterate through new grid
    for new_idx in range(new_size):
        # Get position in new grid
        new_pos = new_grid.index_to_hexposition(new_idx)
        
        # Scale to old grid position (center of sampling region)
        sampling_index = self.hexGrid.hexposition_to_index(
            HexPosition(int(new_pos.q * scale_x), int(new_pos.r * scale_y)),
            self.hexGrid.middle
        )
        
        # Sample using relative offsets (convolution-style!)
        values = []
        valid_weights = []
        for i, offset in enumerate(shape):
            old_idx = self.hexGrid.hexposition_to_index(offset, sampling_index)
            if 0 <= old_idx < len(field):
                values.append(field[old_idx])
                valid_weights.append(weights[i])
        
        # If no valid samples, use 0
        if len(values) == 0:
            new_field[new_idx] = 0
            continue
            
        values = np.array(values)
        valid_weights = np.array(valid_weights)
        
        # Apply aggregation method
        if method == 'weighted_avg':
            new_field[new_idx] = np.average(values, weights=valid_weights)
        elif method == 'max':
            new_field[new_idx] = np.max(values)
        elif method == 'min':
            new_field[new_idx] = np.min(values)
        elif method == 'mode':
            # For categorical data - weighted mode
            unique_vals = np.unique(values)
            best_val = unique_vals[0]
            best_weight = 0
            for val in unique_vals:
                val_weight = np.sum(valid_weights[values == val])
                if val_weight > best_weight:
                    best_weight = val_weight
                    best_val = val
            new_field[new_idx] = best_val
    
    return new_field

## deprecating
@patch
def downsample_field(self: Terrain, field_array, sample_radius=2, method='weighted_avg'):
    """Downsample any field array using hex ring sampling.
    
    Args:
        field_array: numpy array of values to downsample (same length as hexes)
        sample_radius: How many rings to sample
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        Downsampled array matching new grid size
    """
    old_grid = self.hexGrid
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large")
        return field_array
    
    # Calculate new grid size
    new_size = new_rows * new_cols
    new_field = np.zeros(new_size)
    
    for new_idx in range(new_size):
        new_row = new_idx // new_cols
        new_col = new_idx % new_cols
        
        old_row = new_row * factor
        old_col = new_col * factor
        
        if old_row >= old_grid.nRows or old_col >= old_grid.nCols:
            continue
            
        center_idx = old_row * old_grid.nCols + old_col
        
        # Collect values from rings
        values = []
        weights = []
        
        for ring_num in range(sample_radius + 1):
            weight = 1.0 / (ring_num + 1)
            
            if ring_num == 0:
                ring_hexes = [center_idx]
            else:
                ring_positions = HexPosition(0, 0, 0).ring(ring_num)
                ring_hexes = [old_grid.hexposition_to_index(hp, center_idx) 
                             for hp in ring_positions]
                ring_hexes = [h for h in ring_hexes if h >= 0]
            
            for hex_idx in ring_hexes:
                values.append(field_array[hex_idx])
                weights.append(weight)
        
        if not values:
            continue
            
        values = np.array(values)
        weights = np.array(weights)
        
        # Apply aggregation method
        if method == 'weighted_avg':
            new_field[new_idx] = np.average(values, weights=weights)
        elif method == 'max':
            new_field[new_idx] = np.max(values)
        elif method == 'min':
            new_field[new_idx] = np.min(values)
        elif method == 'mode':
            # For categorical data - weighted mode
            unique_vals = np.unique(values)
            best_val = unique_vals[0]
            best_weight = 0
            for val in unique_vals:
                val_weight = np.sum(weights[values == val])
                if val_weight > best_weight:
                    best_weight = val_weight
                    best_val = val
            new_field[new_idx] = best_val
    
    return new_field


#| export
## deprecating
@patch
def downsample_ring(self: Terrain, sample_radius=2, method='weighted_avg'):
    """Downsample terrain using hex ring sampling.
    
    Args:
        sample_radius: How many rings to sample (higher = more smoothing)
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        New Terrain with fewer hexes covering same area
    """
    old_grid = self.hexGrid
    
    # Calculate new grid dimensions
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large for this terrain")
        return self
    
    # Keep same bounds, just use larger hex radius
    new_radius = old_grid.radius * factor
    
    # Create new terrain with same bounds but larger hexes
    new_terrain = Terrain(old_grid.bounds, radius=new_radius)
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    
    # Use the downsample_field method to sample elevations
    new_terrain.elevations = self.downsample_field(
        self.elevations, 
        sample_radius=sample_radius, 
        method=method
    )
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain


In [ ]:
#| export
@patch
def scaled(self: Terrain, scale: float):
    """Create a scaled terrain that maintains proportional grid dimensions"""
    radius = self.hexGrid.radius 

    def hexes_in_range(n):
        """ finding all hexes within distance N from origin: """
        results = []
        for q in range(-n, n + 1):
            for r in range(max(-n, -q - n), min(n, -q + n) + 1):
                s = -q - r
                results.append(HexPosition(q, r, s))
        return results



    ring_pattern = hexes_in_range(1)

    nextEl , nRows, nCols  = self.convolution( self.elevations, ring_pattern, scale)

    grid =  HexGrid (
        nRows = nRows,
        nCols = nCols,
        radius = radius,
        style = self.hexGrid.style)

    mySize = MapSize(400, 400) #we are droping these
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap = Terrain(myBounds, radius=15)
    sampleMap.hexGrid = grid

    sampleMap.elevations = nextEl

    # Copy style properties
    sampleMap.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    sampleMap.elevationDelta = self.elevationDelta
    sampleMap.seaLevel = self.seaLevel

# Use the downsample_field meth

    return sampleMap

    

In [ ]:
@patch
def demoSmall(self:TerraDemo):
    """Practice building up coord."""
    
    sf = self.sanFran()



    mySize = MapSize(400, int(400 * sf.hexGrid.nRows/sf.hexGrid.nCols))
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap = sf.scaled(0.3)

    
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    
    ring_indices = sampleMap.ring(start)
    for idx in ring_indices:
        region.hexes.add(idx)
        
    ring_indices = sampleMap.ring(start,2)
    for idx in ring_indices:
        region.hexes.add(idx)

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    #sampleMap.dot(region.perimeter())
    #sampleMap.fillRegion(region)

    pathstyle=StyleCSS("boundaryPath",fill="none",stroke="#ba3ca3ff",stroke_width=3)
    paths = region.trace_perimeter(style=pathstyle)
    sampleMap.builder.add_style(pathstyle)
    
    
    pathLayer = ""
    for path in paths:
        cl = path.closed()
        w = cl.make_windy(iterations=2, offset_factor=0.2)
        pathLayer += w.svg()
        
    sampleMap.builder.adjust("perimeter_path", pathLayer)
    
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoSmall()

In [ ]:
@patch
def demoSmall(self:TerraDemo):
    """Practice building up coord."""
    
    sf = self.sanFran()
    sampleMap = sf.scaled(0.6)
    
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    
    ring_indices = sampleMap.ring(start)
    for idx in ring_indices:
        region.hexes.add(idx)
        
    sampleMap.hexGrid.adjustRadius(15)
    ring_indices = sampleMap.ring(start,2)
    for idx in ring_indices:
        region.hexes.add(idx)

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    #sampleMap.dot(region.perimeter())
    #sampleMap.fillRegion(region)

    pathstyle=StyleCSS("boundaryPath",fill="none",stroke="#ba3ca3ff",stroke_width=3)
    paths = region.trace_perimeter(style=pathstyle)
    sampleMap.builder.add_style(pathstyle)
    
    
    pathLayer = ""
    for path in paths:
        cl = path.closed()
        w = cl.make_windy(iterations=2, offset_factor=0.2)
        pathLayer += w.svg()
        
    sampleMap.builder.adjust("perimeter_path", pathLayer)
    
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoSmall()

### now lets build a coast

In [ ]:
#| export
@patch
def growFromHex(self: Terrain, center_idx, origin=0):
    """Grow a region from center hex at same elevation level."""
    center_elev_level = self.elevationLevel(center_idx)
    
    # Use a queue-based flood fill instead
    hex_set = {center_idx}
    to_check = [center_idx]
    
    while to_check:
        current = to_check.pop(0)
        
        # Check all 6 neighbors
        neighbors = self.ring(current, 1)
        for n_idx in neighbors:
            if n_idx in hex_set:
                continue
            if n_idx < 0 or n_idx >= len(self.elevations):
                continue
            if self.elevationLevel(n_idx) != center_elev_level:
                continue
            
            hex_set.add(n_idx)
            to_check.append(n_idx)
    
    return HexRegion(hexes=hex_set, hex_grid=self.hexGrid)

In [ ]:
#| export


@patch
def find_region_at_level(self:Terrain, center_idx):
    """Find all connected hexes within tolerance of center_idx's elevation.
    
    Args:
        center_idx: starting hex index
        tolerance: elevation difference allowed
    
    Returns:
        set of hex indices in the region
    """
    target_elev = self.elevationLevel(center_idx)
    levels = [x for x in range(len(self.elevations)-1) if self.elevationLevel(x) == target_elev]

    return set(levels)


In [ ]:
#| export
@patch
def demoRegion(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start =  0 # peaks[0]
    region = sampleMap.growFromHex(start)
    

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    #sampleMap.dot(region.perimeter())
    #sampleMap.fillRegion(region)

    pathstyle=StyleCSS("boundaryPath",fill="none",stroke="#ba3ca3ff",stroke_width=3)
    paths, gaps = region.trace_perimeter(style=pathstyle)
    sampleMap.builder.add_style(pathstyle)
    
    
    pathLayer = ""
    for path in paths:
        cl = path.closed()
        w = cl.make_windy(iterations=2, offset_factor=0.2)
        pathLayer += w.svg()
        
    sampleMap.builder.adjust("perimeter_path", pathLayer)
    
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()


In [ ]:
#| export
@patch
def demoRegion(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start =  0 # peaks[0]
    region = HexRegion(hexes=sampleMap.find_region_at_level(0), hex_grid=sampleMap.hexGrid)
    for x in sampleMap.find_region_at_level(0):
        region.hexes.add(x)
    
    #region = sampleMap.growFromHex(start)
    

    # Show the base terrain
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    #sampleMap.dot(region.perimeter())
    #sampleMap.fillRegion(region)

    pathstyle=StyleCSS("boundaryPath",fill="none",stroke="#ba3ca3ff",stroke_width=3)
    paths = region.trace_perimeter(style=pathstyle)
    sampleMap.builder.add_style(pathstyle)
    
    
    pathLayer = ""
    for path in paths:
        if path.points[0].distance(path.points[-1]) < 40:
            cl = path.closed()
        else:
            cl = path
        w = cl.make_windy(iterations=2, offset_factor=0.2)
        pathLayer += w.svg()
        
    sampleMap.builder.adjust("perimeter_path", pathLayer)
    
    #sampleMap.builder.adjust("root","")
    #print(sampleMap.builder.xml())
    
    return sampleMap.hexGrid.builder.show()
 


In [ ]:

TerraDemo().demoRegion()

In [ ]:
#| export
@patch
def coastline_svg(self:Terrain,pathstyle=StyleCSS("coastPath",fill="none",stroke="#917910ff",stroke_width=3)):
    """Add a coast to the terrain."""
    region = HexRegion(hexes=self.find_region_at_level(0), hex_grid=self.hexGrid)


    for x in self.find_region_at_level(0):
        region.hexes.add(x)
    

    paths = region.trace_perimeter(style=pathstyle,debug=False)
    self.builder.add_style(pathstyle)

    pathLayer = ""
    for path in paths:
        if path.points[0].distance(path.points[-1]) < 40:
            cl = path.closed()
        else:
            cl = path
        curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
        smooth = curved.smooth(iterations=1)
        pathLayer += smooth.svg()
        
    return pathLayer

In [ ]:
#| export
@patch
def addCoast(self:Terrain,pathstyle=StyleCSS("coastPath",fill="none",stroke="#917910ff",stroke_width=3)):
    """Add a coast to the terrain."""
    

    pathLayer = self.coastline_svg(pathstyle=pathstyle)
        
    self.builder.adjust("coastline", pathLayer)
    self.colorMap()
    self.hexGrid.update()

    

In [ ]:
#| export
@patch
def demoCoast(self:TerraDemo):
    """Practice building up coord."""
    
    sampleMap = self.sanFran()
    sampleMap.addCoast()
    sampleMap.builder.layers[0].hide()
    
    return sampleMap.hexGrid.builder.show()
 



In [ ]:
@patch
def demoVolcano(self:TerraDemo):

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=15,path = "volcano.svg")
    
    sampleMap.volcano(center=267,adjusted=500,num_rings=6,variability=0.5,initial_threshold=0.4)
    sampleMap.colorMap()
    sampleMap.addCoast()
    sampleMap.hexGrid.update()
    sampleMap.builder.layers[0].hide()

    return sampleMap.hexGrid.builder.show()


In [ ]:
TerraDemo().demoVolcano()

In [ ]:
TerraDemo().demoCoast()

Future work

In [ ]:
??HexGrid


```python
class HexGrid:
    """Hexagonal grid with cube coordinate support."""

    def __init__(self, 
                 nRows: int,
                 nCols: int, 
                 radius: float,
                 style: StyleCSS,
                 offset: MapCord = None):
        self.nRows = nRows
        self.nCols = nCols
        self.radius = radius
        self.style = style
        self.offset = offset or MapCord(0, 0)

        self.builder = SVGBuilder()
        self.builder.add_style(style)
        for x in StyleCSS.elevations():
            self.builder.add_style(x)

        self._build_hexes()

    def _build_hexes(self):
        """Build hex array from current parameters."""
        self.hexes = []
        for row in range(self.nRows):
            for col in range(self.nCols):
                self.hexes.append(self._hex_at(row, col, self.style))
        self._update_builder_size()

    def _hex_at(self, row: int, col: int, style: StyleCSS) -> Hex:
        """Calculate hex at grid position."""
        width = math.sqrt(3) * self.radius
        height = 2 * self.radius

        # Add radius to offset so first hex center is actually inside the bounds
        x = width * (col + 0.5 * (row % 2)) + self.offset.x + self.radius
        y = height * 0.75 * row + self.offset.y + self.radius

        return Hex(self.radius, MapCord(x, y), style)

    @classmethod
    def from_bounds(cls, bounds: MapRect, radius: float=25, style: StyleCSS=StyleCSS("Hex")) -> 'HexGrid':
        """Create grid to fill a bounding rectangle."""

        nRows = int(bounds.dimensons.height / radius)
        nCols = int(bounds.dimensons.width / radius)
        offset = MapCord(bounds.origin.x - radius, bounds.origin.y - radius)

        return cls(nRows, nCols, radius, style, offset)


    def _update_builder_size(self):
        """Update SVG builder dimensions."""
        if self.hexes:
            self.builder.width = max(h.center.x for h in self.hexes) + self.radius
            self.builder.height = max(h.center.y for h in self.hexes) + self.radius

    def _middle_hex_natural_position(self) -> MapCord:
        """Where the middle hex center would be with zero offset."""
        middle_row = self.nRows // 2
        middle_col = self.nCols // 2

        width = math.sqrt(3) * self.radius
        height = 2 * self.radius

        x = width * (middle_col + 0.5 * (middle_row % 2))
        y = height * 0.75 * middle_row

        return MapCord(x, y)

    def adjustRadius(self, new_radius: float):
        """Adjust radius while keeping middle hex at same pixel position."""
        if not self.hexes:
            self.radius = new_radius
            return

        # Get current middle hex pixel position
        middle_idx = self.middle
        old_middle_pos = self.hexes[middle_idx].center

        # Update radius
        self.radius = new_radius

        # Calculate where middle would be with zero offset
        natural_middle = self._middle_hex_natural_position()

        # Offset to keep middle at same pixel position
        self.offset = MapCord(
            old_middle_pos.x - natural_middle.x + self.offset.x,
            old_middle_pos.y - natural_middle.y + self.offset.y
        )

        # Rebuild hexes with new radius and offset
        self._build_hexes()



    @classmethod
    def centered(cls, rings: int, radius: float, style: StyleCSS, 
                 center: MapCord = None) -> 'HexGrid':
        """Create grid with specified rings around a center point."""
        n = 2 * rings + 1

        # Create grid first with no offset
        grid = cls(nRows=n, nCols=n, radius=radius, style=style, offset=MapCord(0, 0))

        if center is not None:
            # Calculate offset to place middle hex at center
            natural_middle = grid._middle_hex_natural_position()
            grid.offset = MapCord(
                center.x - natural_middle.x,
                center.y - natural_middle.y
            )
            grid._build_hexes()

        return grid

    @property
    def middle(self) -> int:
        """Middle hex index."""
        return (self.nRows // 2) * self.nCols + (self.nCols // 2)

    @property
    def bounds(self) -> MapRect:
        """Bounding rectangle of grid."""
        if not self.hexes:
            return MapRect(MapCord(0, 0), MapSize(0, 0))
        return MapRect(
            self.hexes[0].center,
            MapSize(self.nCols * self.radius * math.sqrt(3), 
                    self.nRows * self.radius * 1.5)
        )


    def text(self, cb=lambda s, i: i):
        i = 0
        for row in range(self.nRows):
            line = "|"
            for col in range(self.nCols):
                line += f" {cb(self, i):5}"
                i += 1
            print(line + " |")

    def rowPartity(self, index):
        "This returns whether a row is even or odd"
        return int(index / self.nCols) % 2

    @property
    def midpoint(self):
        return int(len(self.hexes)/2)
```

**File:** `~/HexMagic/HexMagic/primitives.py`